In [1]:
# !pip install tqdm

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import datetime as dt

# from sort_dataframeby_monthorweek import *
# from tqdm.notebook import tqdm
# import gc

print('imported library')

imported library


In [3]:
from sklearn.preprocessing import LabelEncoder


from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression,SGDRegressor,ElasticNet,Ridge
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestRegressor

from sklearn import linear_model

# from sklearn.pipeline import make_pipeline
# from sklearn.preprocessing import StandardScaler

from sklearn.metrics import mean_squared_error,mean_absolute_error , r2_score

# from sklearn  import metrics


# Reading DataSet

In [4]:

url = '..\favorita-grocery-sales-forecasting'

train = pd.read_csv(os.path.join(url, "train.csv") , nrows=1000000 , parse_dates = ['date'] , index_col = 'id' )
# train = pd.read_csv(os.path.join(url, "train.csv") ,skiprows=120000000 , nrows=1000000 , parse_dates = ['date'] , index_col = 'id')

test = pd.read_csv(os.path.join(url,"test.csv")  , parse_dates = ['date'] , index_col = 'id')

# sample_sub = pd.read_csv(os.path.join(url,"sample_submission.csv"))

stores = pd.read_csv(os.path.join(url,"stores.csv"))

items = pd.read_csv(os.path.join(url,"items.csv"))

transactions = pd.read_csv(os.path.join(url,"transactions.csv") , parse_dates = ['date'])

oil = pd.read_csv(os.path.join(url,"oil.csv") , parse_dates = ['date'])

holiday = pd.read_csv(os.path.join(url,"holidays_events.csv") , parse_dates = ['date'])

print('all of dataset loaded')


C:\Users\SG\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


all of dataset loaded


# Pre_processing :

In [5]:
def catgirical_numeric (df) :
    
    # split name of categorical columns
    cols = df.columns
    num_cols = df._get_numeric_data().columns
    cat_cols = list(set(cols) - set(num_cols))
    print(cat_cols)
    
    # Transform categorical to numerical
    for col in cat_cols :
        encoder = LabelEncoder()
        df[col] = encoder.fit_transform(df[col].values)
          
    return df     

In [6]:
# items 1 , stores  3  , holiday 4  ,

catgirical_numeric(items)
catgirical_numeric(stores)

copy_holiy = holiday.copy()
holiday = catgirical_numeric(holiday.drop('date' , axis = 1))
#  چون تاریخ به عنوان غیر عدد در نظر میگیرد اینجا حذفش میکنم
holiday['date'] = copy_holiy['date']

['family']
['city', 'type', 'state']
['locale_name', 'description', 'locale', 'type']


In [7]:
def distributed(df):
    
    transformed=[]

    # for value in tqdm(train.unit_sales):
    for value in df.unit_sales:
        if float(value) > 0:
            value=np.log1p(float(value))
        else:
            value=0
        transformed.append(value)

    df['unit_sales'] = transformed  
    del transformed

# ----------------------------------------------------------------------

def pre_Train(df) :
    
    df['onpromotion'].fillna( 0 , inplace = True )  #.astype(int)
    df['onpromotion'] = df['onpromotion'].astype(int)

# ----------------------------------------------------------------------
    
pre_Train(train)
distributed(train)

In [8]:
def merge_data():
       
    train_items = train.merge(items , left_on='item_nbr' , right_on ='item_nbr' )
    train_items_stores = train_items.merge(stores , left_on='store_nbr' , right_on='store_nbr')
    train_items_stores_oil = train_items_stores.merge(oil , how ='left' ,  left_on='date' , right_on='date')
    train_items_stores_oil_holiday = train_items_stores_oil.merge(holiday , how ='left' , left_on='date' , right_on='date')
    train_items_stores_oil_holiday_transaction = train_items_stores_oil_holiday .merge(transactions , left_on=['date', 'store_nbr'] , right_on=['date', 'store_nbr'])
    
    return train_items_stores_oil_holiday_transaction


df = merge_data()

print('df shape : ' , df.shape)
print ( 'columns => ' , df.columns )
print ( 'types   => ' , df.dtypes )

df.head()

df shape :  (1000000, 19)
columns =>  Index(['date', 'store_nbr', 'item_nbr', 'unit_sales', 'onpromotion', 'family',
       'class', 'perishable', 'city', 'state', 'type_x', 'cluster',
       'dcoilwtico', 'type_y', 'locale', 'locale_name', 'description',
       'transferred', 'transactions'],
      dtype='object')
types   =>  date            datetime64[ns]
store_nbr                int64
item_nbr                 int64
unit_sales             float64
onpromotion              int32
family                   int32
class                    int64
perishable               int64
city                     int32
state                    int32
type_x                   int32
cluster                  int64
dcoilwtico             float64
type_y                 float64
locale                 float64
locale_name            float64
description            float64
transferred             object
transactions             int64
dtype: object


,date,store_nbr,item_nbr,unit_sales,onpromotion,family,class,perishable,city,state,type_x,cluster,dcoilwtico,type_y,locale,locale_name,description,transferred,transactions
0,2013-01-01,25,103665,2.079442,0,5,2712,1,20,13,3,1,NaN,3.0,1.0,4.0,51.0,False,770
1,2013-01-01,25,105574,0.693147,0,12,1045,0,20,13,3,1,NaN,3.0,1.0,4.0,51.0,False,770
2,2013-01-01,25,105575,1.098612,0,12,1045,0,20,13,3,1,NaN,3.0,1.0,4.0,51.0,False,770
3,2013-01-01,25,108079,0.693147,0,12,1030,0,20,13,3,1,NaN,3.0,1.0,4.0,51.0,False,770
4,2013-01-01,25,108701,0.693147,0,9,2644,1,20,13,3,1,NaN,3.0,1.0,4.0,51.0,False,770


In [9]:
df.shape

(1000000, 19)

In [10]:
def prepare_data (data_df) :
        
#       data_df.isnull().sum()
        data_df.fillna( 0 , axis=1 , inplace = True )
        
#       data_df['dcoilwtico'].fillna( 0 , inplace = True )
        data_df['dcoilwtico'] = data_df['dcoilwtico'].astype(int)
        
     # bool to int
        data_df['transferred'] = data_df['transferred'].astype(int)
                
        # handle date
        data_df['date'] = pd.to_datetime(data_df['date'])
        data_df['date'] = data_df['date'].map(dt.datetime.toordinal)
        
        return data_df

        
data = prepare_data(df)
data.head()

,date,store_nbr,item_nbr,unit_sales,onpromotion,family,class,perishable,city,state,type_x,cluster,dcoilwtico,type_y,locale,locale_name,description,transferred,transactions
0,734869,25,103665,2.079442,0,5,2712,1,20,13,3,1,0,3.0,1.0,4.0,51.0,0,770
1,734869,25,105574,0.693147,0,12,1045,0,20,13,3,1,0,3.0,1.0,4.0,51.0,0,770
2,734869,25,105575,1.098612,0,12,1045,0,20,13,3,1,0,3.0,1.0,4.0,51.0,0,770
3,734869,25,108079,0.693147,0,12,1030,0,20,13,3,1,0,3.0,1.0,4.0,51.0,0,770
4,734869,25,108701,0.693147,0,9,2644,1,20,13,3,1,0,3.0,1.0,4.0,51.0,0,770


In [11]:
data.shape

(1000000, 19)

# split data according to our feature :

In [12]:
# X = data.drop(['unit_sales', 'transactions'], axis=1)
# Y = data[['unit_sales', 'transactions']]

X = data.drop('unit_sales', axis=1)
Y = data['unit_sales']

In [13]:
X.shape   # (578, 18)      (5000000, 18)
Y.shape  # (578,)        (5000000,)

(1000000,)

# Generate test and training data :

In [14]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

In [15]:
x_train.shape  # (404, 18)

(700000, 18)

In [16]:
y_train.shape  # (404,)

(700000,)

In [17]:
x_test.shape  # (174, 18)

(300000, 18)

In [18]:
y_test.shape  # (174,)

(300000,)

# Modelling & testing :

In [19]:
def checkModelPerformane(model):
    
    m = model.fit(x_train, y_train)
    pred = model.predict(x_test.values)
    
    print("mean_squared_error  =>  ",  np.sqrt(mean_squared_error(y_test.values, pred))) 
    print("mean_absolute_error =>  ",  np.sqrt(mean_absolute_error(y_test.values, pred)))
    
    print("r2_score =>  ",  r2_score(y_test, pred) )
#   print('score    =>  ',  m.score(x_test,y_test) ) 
    
    return m


# a = y_test.to_frame()
# a['pred'] = pred
# a

# Linear regression :

In [20]:
print("LinearRegression")
checkModelPerformane(LinearRegression())

LinearRegression
mean_squared_error  =>   0.8223129742781861
mean_absolute_error =>   0.8115437206686469
r2_score =>   0.11530526158265975


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

# Lasso regression :

In [21]:
print("lasso regression ")
checkModelPerformane(linear_model.Lasso(alpha=0.1))

lasso regression 
mean_squared_error  =>   0.8253777762885367
mean_absolute_error =>   0.8131648299907763
r2_score =>   0.10869836837602309


Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

# ElasticNet regression :

In [22]:
print("ElasticNet regression ")
checkModelPerformane(ElasticNet())

ElasticNet regression 
mean_squared_error  =>   0.8266500141997827
mean_absolute_error =>   0.8138387690978277
r2_score =>   0.10594854473735005


ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.5,
           max_iter=1000, normalize=False, positive=False, precompute=False,
           random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

# Ridge regression :

In [23]:
print("Ridge regression ")
checkModelPerformane(Ridge(alpha=1.0))

Ridge regression 
mean_squared_error  =>   0.8223129017285985
mean_absolute_error =>   0.8115437148971971
r2_score =>   0.11530541768924574


C:\Users\SG\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=1.31828e-17): result may not be accurate.
  overwrite_a=True).T


Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

# Random forests :

In [24]:
print("Random Forest")
rf = checkModelPerformane(RandomForestRegressor(random_state=42))
rf

C:\Users\SG\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Random Forest
mean_squared_error  =>   0.5534487160507352
mean_absolute_error =>   0.6578070086870773
r2_score =>   0.5992496338122253


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=42, verbose=0,
                      warm_start=False)

In [25]:
from pprint import pprint# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(rf.get_params())

Parameters currently in use:

{'bootstrap': True,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 10,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}


    n_estimators = number of trees in the foreset
    max_features = max number of features considered for splitting a node
    max_depth = max number of levels in each decision tree
    min_samples_split = min number of data points placed in a node before the node is split
    min_samples_leaf = min number of data points allowed in a leaf node
    bootstrap = method for sampling data points (with or without replacement)

